In [3]:
from functions.test_algorithms import *
from functions.random_search import * 
from slim_gsgp_lib.datasets.data_loader import *
import pickle

datasets = [globals()[i] for i in globals() if 'load' in i][2:]

#### Settings

In [13]:
pop_size = 100 
n_iter = 100
n_iter_rs = 50
n_iter_test = 30
p_train = 0.7

#### SLIM 

In [ ]:
for dataset_loader in tqdm(datasets):
    X, y = dataset_loader()
    dataset_name = dataset_loader.__name__.split('load_')[1]

    # Test if the best hyperparameters are valid
    try: 
        with open(f"best_params/best_slim_{dataset_name}_{pop_size}_{n_iter}_True_just_strmut.pkl", 'rb') as f:
            best_params = pickle.load(f)
            print(f"Best hyperparameters for {dataset_name} already exist!")
    except:
        best_params = {}
            
    # Perform random search for both scaled and unscaled versions
    if best_params == {}:
        print(f"Performing random search for {dataset_name}...")
        best_params = random_search_slim(X, y, dataset_name, scale=True, p_train=p_train,
                                            iterations=n_iter_rs, pop_size=pop_size, n_iter=n_iter,
                                            struct_mutation=True, show_progress=False, x_o=False, identifier='just_strmut')
        
        print(f"Random search for {dataset_name} completed!")
    
    # Initialize dictionaries for scaled and unscaled results
    metrics = ['rmse_', 'mape_', 'nrmse_', 'r2_', 'mae_', 'std_rmse_', 'time_stats', 'train_fit', 'test_fit', 'size']
    results = {metric: {} for metric in metrics}

    for algorithm in best_params:
        # Retrieve the best hyperparameters for testing
        args = best_params[algorithm]

        # Test SLIM 
        rm, ma, nrmse, r2, mae, std_rmse, time, train, test, size = test_slim(
            X=X, y=y, args_dict=args, dataset_name=dataset_loader.__name__,
            ms_lower=0, ms_upper=1, n_elites=1,
            iterations=n_iter_test, scale=True, algorithm=algorithm,
            verbose=0, p_train=p_train, show_progress=False,
        )
        
        # Initialize storage for each algorithm if not already present
        for metric in metrics:
            if algorithm not in results[metric]:
                results[metric][algorithm] = []

        # Store results
        results['rmse_'][algorithm].append(rm)
        results['mape_'][algorithm].append(ma)
        results['nrmse_'][algorithm].append(nrmse)
        results['r2_'][algorithm].append(r2)
        results['mae_'][algorithm].append(mae)
        results['std_rmse_'][algorithm].append(std_rmse)
        results['time_stats'][algorithm].append(time)
        results['train_fit'][algorithm].append(train)
        results['test_fit'][algorithm].append(test)
        results['size'][algorithm].append(size)        

        print(f"Results for {algorithm} on {dataset_name} saved!")

    # Save the results to disk
    with open(f"results/SLIM/{dataset_name}_just_strmut.pkl", 'wb') as f:
        pickle.dump(results, f)

    print(f"Results for {dataset_name} saved!")
    print("---------------------------------------------------")

  0%|          | 0/16 [00:00<?, ?it/s]

Best hyperparameters for resid_build_sale_price already exist!
Results for SLIM+SIG2 on resid_build_sale_price saved!
Results for SLIM*SIG2 on resid_build_sale_price saved!
Results for SLIM+ABS on resid_build_sale_price saved!
Results for SLIM*ABS on resid_build_sale_price saved!
Results for SLIM+SIG1 on resid_build_sale_price saved!


  6%|▋         | 1/16 [20:35<5:08:53, 1235.58s/it]

Results for SLIM*SIG1 on resid_build_sale_price saved!
Results for resid_build_sale_price saved!
---------------------------------------------------
Performing random search for istanbul...
Random search for istanbul completed!
Results for SLIM+SIG2 on istanbul saved!
Results for SLIM*SIG2 on istanbul saved!
Results for SLIM+ABS on istanbul saved!
Results for SLIM*ABS on istanbul saved!
Results for SLIM+SIG1 on istanbul saved!


 12%|█▎        | 2/16 [1:07:57<8:28:45, 2180.36s/it]

...

Results for SLIM*SIG1 on efficiency_heating saved!
Results for efficiency_heating saved!
---------------------------------------------------
Performing random search for efficiency_cooling...
Random search for efficiency_cooling completed!
Results for SLIM+SIG2 on efficiency_cooling saved!
Results for SLIM*SIG2 on efficiency_cooling saved!
Results for SLIM+ABS on efficiency_cooling saved!
Results for SLIM*ABS on efficiency_cooling saved!
Results for SLIM+SIG1 on efficiency_cooling saved!


 69%|██████▉   | 11/16 [8:11:13<3:52:35, 2791.04s/it]

Results for SLIM*SIG1 on efficiency_cooling saved!
Results for efficiency_cooling saved!
---------------------------------------------------
Performing random search for forest_fires...
Random search for forest_fires completed!
Results for SLIM+SIG2 on forest_fires saved!
Results for SLIM*SIG2 on forest_fires saved!
Results for SLIM+ABS on forest_fires saved!
Results for SLIM*ABS on forest_fires saved!
Results for SLIM+SIG1 on forest_fires saved!


 75%|███████▌  | 12/16 [8:58:25<3:06:53, 2803.45s/it]

Results for SLIM*SIG1 on forest_fires saved!
Results for forest_fires saved!
---------------------------------------------------
Performing random search for parkinson_updrs...
